# Realiza a transformação do dataset
Extrai os seguintes dados 'data', 'relator', 'orgao', 'recorrente', 'dispositivo', 'qtd_recorrentes'

## Importação de bibliotecas básicas

In [14]:
import pandas as pd
import time

## Carregamento do dataset

In [15]:
pasta = "E:\\OneDrive\\Documentos\\pesquisa_codigos_2020\\pesquisa_jurimetria_polaridade_final\\2_Compreensao_dos_dados\\2_1_coleta_de_dados\\Datasets_coletados\\TRT3\\"
arquivo = "TRT3_inteiro_teor_2017_2018_2019_amostra_2.csv"
df = pd.read_csv( pasta+arquivo, encoding='utf-8' )
print( df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8787 entries, 0 to 8786
Data columns (total 1 columns):
INTEIRO_TEOR    8786 non-null object
dtypes: object(1)
memory usage: 68.8+ KB
None


In [16]:
#df = df.drop( "_type", 1)
#df.columns = [ "LINK", "INTEIRO_TEOR" ]
#df.info()

In [17]:
df.head()

,INTEIRO_TEOR
0,PJe: 0010502-26.2018.5.03.0012 (RO) Dispon...
1,PJe: 0057800-17.2007.5.03.0071 (AP) Dispon...
2,PJe: 0011246-83.2017.5.03.0035 (AP) Dispon...
3,PJe: 0001927-54.2012.5.03.0104 (AP) Dispon...
4,PJe: 0011536-13.2016.5.03.0010 (RO) Dispon...


## Definição de funções para extrair os dados

In [18]:
import re

def extrai_data( texto ):
    try:
        result = re.search( r"(\d{1,2}\/\d{1,2}\/(\d{4}|\d{2}))", str( texto ) )
        return result[1]
    except:
        return pd.np.nan

def extrai_relator( texto ):
    try:
        regex1 = r"relator:|redator:"
        achou = re.search(regex1, texto.lower() )
        inicio = achou.end()

        regex2 = r"ementa|órgão|poder"
        achou = re.search(regex2, texto[inicio:].lower())
        fim = achou.start()
        return texto[inicio:inicio+fim].strip()
    except:
        return pd.np.nan

def extrai_orgao( texto ):
    try:
        regex1 = r"órgão julgador:|órgão julgador:"
        achou = re.search( regex1, texto.lower() )
        inicio = achou.end()

        regex2 = r"data:|relator|redator"
        achou = re.search( regex2, texto[inicio:].lower() )
        fim = achou.start()

        orgao = texto[inicio:inicio+fim].strip()
        return orgao
    except:
        return pd.np.nan
    
def extrai_recorrente(texto):
    try:
        regex1 = r"recorrentes:|recorrente:|agravante:|agravantes:|embargante:|embargantes:"
        achou = re.search( regex1, texto.lower() )
        inicio = achou.end()

        #regex2 = r"recorridos:|recorrido:|agravado:|agravados:|agravada:|agravadas:|recorridas:|recorrida:|representante:|terceiro interessado:|relator:|redator:"
        regex2 = r"recorridos|recorrido|agravado|agravados|agravada|agravadas|recorridas|recorrida|representante|terceiro interessado|relator|redator|embargado|embargados"
        achou = re.search( regex2, texto[inicio:].lower() )
        fim = achou.start()

        recorrente = texto[inicio:inicio+fim].strip()
        return recorrente
    except:
        return pd.np.nan

def extrai_dispositivo(texto):
    try:
        regex1 = r"ACORDAM|ACÓRDÃO|ACORDAO"
        achou = re.search( regex1, texto ) #Sem o .lower() pq ta sempre em maiúsculo
        
        if achou == None: #se nao achar, maiúsculo, tenta minúsculo
            regex1 = r"acordam|acórdão|acordao|fundamentos pelos quais"
            achou = re.search( regex1, texto.lower() )        
        
        inicio = achou.end()

        regex2 = r"fundamentos pelos quais"
        achou = re.search( regex2, texto[inicio:].lower() )
        if achou != None:  #se ja achou o inicio, e achou outra palavra fundamentos, vai ate o final
            return texto[inicio:].strip()
        
        regex2 = r"relatório|fundamentos"
        achou = re.search( regex2, texto[inicio:].lower() )
        if achou != None:
            fim = achou.start()
            return texto[inicio:inicio+fim].strip()
        else:
            return texto[inicio:].strip()
    except:
        return pd.np.nan
    
def quantidade_recorrentes( texto ):
    try:
        s = texto.split(",")
        if len( s ) == 1 and len( texto ) > 30: # tem nome q nao tem ",", daí se for maior q 30 considero q sao 2 nomes
            return 999
        else:
            return int( len( s ) )
    except:
        return pd.np.nan

## Extração das informações

In [19]:
%%time
df_tudo=df
#df_tudo.reset_index()
df_tudo["DATA"] = df_tudo["INTEIRO_TEOR"].map( extrai_data )
df_tudo["RELATOR"] = df_tudo["INTEIRO_TEOR"].map( extrai_relator )
df_tudo["ORGAO"] = df_tudo["INTEIRO_TEOR"].map( extrai_orgao )
df_tudo["RECORRENTE"] = df_tudo["INTEIRO_TEOR"].map( extrai_recorrente )
df_tudo["DISPOSITIVO"] = df_tudo["INTEIRO_TEOR"].map( extrai_dispositivo )
df_tudo["QTD_RECORRENTES"] = df_tudo["RECORRENTE"].map( quantidade_recorrentes )

Wall time: 4.83 s


In [20]:
df_tudo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8787 entries, 0 to 8786
Data columns (total 7 columns):
INTEIRO_TEOR       8786 non-null object
DATA               8779 non-null object
RELATOR            8784 non-null object
ORGAO              8784 non-null object
RECORRENTE         8655 non-null object
DISPOSITIVO        8784 non-null object
QTD_RECORRENTES    8655 non-null float64
dtypes: float64(1), object(6)
memory usage: 480.7+ KB


## Limpeza dos dados

In [21]:
#sem html e sem NAs
#apenas 1 recorrente
df_tudo = df_tudo.dropna()
df_tudo["QTD_RECORRENTES"].unique()

array([  1., 999.,   2.,   3.,   5.,  38.,  27.,   4.,  19.,  59.,  12.,
        55.,   7.,  13.,  39.,   6.,  21.,  16., 138.,  23.,   8.,  17.,
        28.,  11.,  32.,  25.,  66.,  56.,  36.,  18.,  10.,  29.,  61.,
        43.,  31.,  68.,  49.,  41.,  64.,  78.,  37., 141.,  33.,  34.,
        50.,  30.,  35.,  47.,  57.,  22.,  44.,  80.,  51.,   9.,  94.,
        40.,  26.,  74.,  69., 289.,  60., 108.,  54.,  15.,  65.])

In [22]:
df_tudo["QTD_RECORRENTES"].value_counts() 

1.0      3585
999.0    3276
2.0      1266
3.0       219
4.0        89
         ... 
51.0        1
31.0        1
54.0        1
33.0        1
59.0        1
Name: QTD_RECORRENTES, Length: 65, dtype: int64

In [23]:
df_tudo.loc[ df_tudo["QTD_RECORRENTES"] == 1, : ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3585 entries, 0 to 8786
Data columns (total 7 columns):
INTEIRO_TEOR       3585 non-null object
DATA               3585 non-null object
RELATOR            3585 non-null object
ORGAO              3585 non-null object
RECORRENTE         3585 non-null object
DISPOSITIVO        3585 non-null object
QTD_RECORRENTES    3585 non-null float64
dtypes: float64(1), object(6)
memory usage: 224.1+ KB


In [24]:
df_filtrada = df_tudo.loc[ df_tudo["QTD_RECORRENTES"] == 1, : ]
df_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3585 entries, 0 to 8786
Data columns (total 7 columns):
INTEIRO_TEOR       3585 non-null object
DATA               3585 non-null object
RELATOR            3585 non-null object
ORGAO              3585 non-null object
RECORRENTE         3585 non-null object
DISPOSITIVO        3585 non-null object
QTD_RECORRENTES    3585 non-null float64
dtypes: float64(1), object(6)
memory usage: 224.1+ KB


## Salvamento dos dados

In [25]:
pasta = ".\\Datasets_transformados\\"
arquivo = arquivo[:-4]+"_transf.csv"
df_filtrada.to_csv( pasta+arquivo, encoding='utf-8', index=False )